In [1]:
import time
import sys

import numpy as np
from pandas.io.json import json_normalize
from tqdm import tqdm_notebook

import torch
import torch.optim as optim
import torch.nn.functional as F
from cnn import make_cifar10_dataset_resnet, make_model, download_cifar10

In [2]:
DATA = "./data"
BATCH_SIZE = 128
max_cpu_count = 4
do_transform = False

In [3]:
download_time, untar_time = download_cifar10(DATA)
print("  download time:", round(download_time, 3))
print("  untar time:", round(untar_time, 3))

  download time: -1
  untar time: -1


In [9]:
def add_item(stats, bench, do_transform, device, workers, 
             model, time, batches, images):
    stats.append({})
    stats[-1]["benchmark"] = bench
    stats[-1]["transform"] = do_transform
    stats[-1]["workers"] = workers
    stats[-1]["device"] = device
    stats[-1]["model"] = model
    stats[-1]["time"] = time
    stats[-1]["batches"] = batches
    stats[-1]["images/sec"] = images
    
stats = []

In [10]:
cuda_devices = list(range(torch.cuda.device_count()))
model_list = ["resnet18", "resnet152"]

for cuda_device in cuda_devices:
    for model_name in model_list:
        for num_workers in range(1):
            print("Device:\t", cuda_device)
            print("Model:\t", model_name)
            print("Workers:\t", num_workers)
            trn_loader = make_cifar10_dataset_resnet(DATA, BATCH_SIZE, device=cuda_device, transformations=do_transform,
                                                     distributed=False, num_workers=num_workers)

            model = make_model(model_name).to(cuda_device)
            optimizer = optim.Adam(model.parameters())
            
            start = time.time()
            for batch_i, (batch, labels) in enumerate(trn_loader):
                optimizer.zero_grad()
                batch, labels = batch.to(cuda_device), labels.to(cuda_device)
                preds = model(batch)
                loss = F.cross_entropy(preds, labels)
                loss.backward()
                optimizer.step()
            model_time = time.time() - start
            add_item(stats, "cifar", do_transform, cuda_device, num_workers, 
                     model_name, model_time, batch_i, BATCH_SIZE * batch_i / model_time)
            print()

Device:	 0
Model:	 resnet18
Workers:	 0

Device:	 0
Model:	 resnet152
Workers:	 0



In [20]:
df = json_normalize(stats)
print(df)
df.to_csv("./logs.txt")

   batches benchmark  device   images/sec      model        time  workers
0      390     cifar       0  1140.399378   resnet18   43.774138        0
1      390     cifar       0   485.805311  resnet152  102.757213        0


In [11]:
df = json_normalize(stats)
print(df)
df.to_csv("./logs.txt")

   batches benchmark  device   images/sec      model       time  transform  \
0      390     cifar       0  1920.262599   resnet18  25.996444      False   
1      390     cifar       0   542.429185  resnet152  92.030446      False   

   workers  
0        0  
1        0  
